In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import requests
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [143]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Belo_Horizonte'
WikiBH = requests.get(url).content
soup = bs.BeautifulSoup(WikiBH,'html.parser')     

In [144]:
data = [];
for item in range(len(soup.find_all('tbody')[3].find_all('tr'))):
    row=[]
    if item >0:
        for row_item in range(len(soup.find_all('tbody')[3].find_all('tr')[1].find_all('td'))):
            row.append(soup.find_all('tbody')[3].find_all('tr')[item].find_all('td')[row_item].get_text())
    data.append(row)

df = pd.DataFrame(data,columns=['#','Zone','Regional','Neighborhood','Population','Size(km²)'])
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df

,#,Zone,Regional,Neighborhood,Population,Size(km²)
0,1,ZONA LESTE,Leste,Sagrada Família,34 395,"2,291\n"
1,2,ZONA SUL,Oeste,Buritis,29 374,"3,823\n"
2,3,ZONA OESTE,Noroeste,Padre Eustáquio,28 773,"2,969\n"
3,4,ZONA OESTE,Barreiro,Lindéia,25 231,"2,012\n"
4,5,ZONA NORTE,Pamp./ Venda Nova,Santa Mônica,23 883,"2,178\n"
...,...,...,...,...,...,...
498,483,ZONA LESTE,Nordeste,Guanabara,0,"0,002\n"
499,484,ZONA NORTE,Pampulha,Lagoa da Pampulha,0,"3,138\n"
500,485,ZONA NORTE,Noroeste,Sumaré,0,"0,149\n"
501,486,ZONA LESTE,Leste,Vila Vera Cruz I,0,"0,008\n"


In [145]:
df['latitude']=0.0
df['longitude']=0.0
for i in range(len(df['Neighborhood'])):
    address = df['Neighborhood'][i] +', Belo Horizonte'                                
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    if location != None:
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.  {}'.format(address,latitude, longitude,i))
        df['latitude'][i]=latitude
        df['longitude'][i]=longitude
    else:
        print(df['Neighborhood'][i] + ' wasn\'t found on Nominatim')
        df['latitude'][i]=''
        df['longitude'][i]=''

The geograpical coordinate of Sagrada Família, Belo Horizonte are -20.1964041, -44.1078058.  0


<ipython-input-145-123152a0de33>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'][i]=latitude
<ipython-input-145-123152a0de33>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'][i]=longitude


The geograpical coordinate of Buritis, Belo Horizonte are -19.9765791, -43.9674165.  1
The geograpical coordinate of Padre Eustáquio, Belo Horizonte are -19.9143045, -43.97935599013762.  2
The geograpical coordinate of Lindéia, Belo Horizonte are -19.9774137, -44.0506354.  3
The geograpical coordinate of Santa Mônica, Belo Horizonte are -19.8245686, -43.977456.  4
The geograpical coordinate of Céu Azul, Belo Horizonte are -19.8227499, -44.0039799.  5
The geograpical coordinate of Santa Cruz (ZN), Belo Horizonte are -20.1377829, -44.2008425.  6
The geograpical coordinate of Santa Cruz (ZL), Belo Horizonte are -20.1377829, -44.2008425.  7
The geograpical coordinate of Santo Antônio, Belo Horizonte are -19.4755851, -44.2603603.  8
The geograpical coordinate of Alto Vera Cruz, Belo Horizonte are -19.9110402, -43.8915918.  9
The geograpical coordinate of Jardim dos Comerciários, Belo Horizonte are -19.7927251, -43.9720853.  10
The geograpical coordinate of Piratininga, Belo Horizonte are -1

The geograpical coordinate of Itapoã, Belo Horizonte are -19.8385689, -43.9554151.  89
The geograpical coordinate of Jardim Leblon, Belo Horizonte are -19.8236179, -43.9913595.  90
The geograpical coordinate of Cachoeirinha, Belo Horizonte are -19.8902003, -43.9474556.  91
The geograpical coordinate of Bairro das Indústrias I, Belo Horizonte are -19.963364, -44.0029681.  92
The geograpical coordinate of Santa Inês, Belo Horizonte are -19.8903311, -43.9096812.  93
The geograpical coordinate of Lagoa, Belo Horizonte are -19.8088758, -44.0001975.  94
The geograpical coordinate of São Bernardo, Belo Horizonte are -19.8449229, -43.941626.  95
The geograpical coordinate of Renascença, Belo Horizonte are -19.8912995, -43.9417132.  96
The geograpical coordinate of Dona Clara, Belo Horizonte are -19.859062, -43.942463.  97
The geograpical coordinate of Funcionários, Belo Horizonte are -19.932516200000002, -43.92731827346328.  98
The geograpical coordinate of Belvedere, Belo Horizonte are -19.97

<ipython-input-145-123152a0de33>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'][i]=''
C:\Users\lucas\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-145-123152a0de33>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'][i]=''


The geograpical coordinate of Horto Florestal, Belo Horizonte are -19.89546035, -43.91695737553633.  156
The geograpical coordinate of Jardim Montanhês, Belo Horizonte are -19.9051254, -43.9821385.  157
The geograpical coordinate of Juliana, Belo Horizonte are -19.8151, -43.9439193.  158
The geograpical coordinate of São Salvador, Belo Horizonte are -19.8939304, -44.0133336.  159
The geograpical coordinate of Coração de Jesus, Belo Horizonte are -19.945874099999997, -43.95122862577084.  160
The geograpical coordinate of Silveira, Belo Horizonte are -19.8917944, -43.9299132.  161
The geograpical coordinate of Olaria, Belo Horizonte are -19.9908908, -44.0277945.  162
The geograpical coordinate of Pompéia, Belo Horizonte are -19.9096481, -43.9024279.  163
The geograpical coordinate of Vila CEMIG, Belo Horizonte are -19.995432800000003, -43.992366573805114.  164
The geograpical coordinate of Nova Cintra, Belo Horizonte are -19.9480576, -43.9900717.  165
The geograpical coordinate of Castan

The geograpical coordinate of Barroca, Belo Horizonte are -19.931433, -43.9635526.  242
The geograpical coordinate of Dom Joaquim, Belo Horizonte are -19.8781302, -43.9188798.  243
The geograpical coordinate of Pirajá, Belo Horizonte are -19.8659435, -43.9170773.  244
The geograpical coordinate of Conjunto Paulo VI, Belo Horizonte are -19.8314739, -43.8865391138505.  245
The geograpical coordinate of Alto dos Pinheiros, Belo Horizonte are -19.9325674, -44.004875.  246
The geograpical coordinate of Parque São José, Belo Horizonte are -19.9625893, -43.9779282.  247
The geograpical coordinate of Vila Califórnia, Belo Horizonte are -19.9189875, -44.00829.  248
The geograpical coordinate of Novo Tupi, Belo Horizonte are -19.8306232, -43.9048449.  249
The geograpical coordinate of Vila Independência I, Belo Horizonte are -29.5821991, -51.8229041.  250
The geograpical coordinate of Conjunto Jardim Filadélfia, Belo Horizonte are -19.9107907, -44.007685.  251
The geograpical coordinate of São L

The geograpical coordinate of Vila Coqueiral, Belo Horizonte are -19.9152291, -44.0247224.  329
The geograpical coordinate of Cidade Jardim, Belo Horizonte are -19.8766162, -43.6567339.  330
The geograpical coordinate of Vila Trinta e Um de Março, Belo Horizonte are -19.9204149, -44.0009698.  331
The geograpical coordinate of São José, Belo Horizonte are -19.8602115, -43.9656973.  332
The geograpical coordinate of Solimões, Belo Horizonte are -4.3941661, -69.8154378.  333
The geograpical coordinate of Bernadete, Belo Horizonte are -19.985184099999998, -43.98424361198627.  334
The geograpical coordinate of Xodó-Marize, Belo Horizonte are -19.821801649999998, -43.94017477200708.  335
The geograpical coordinate of Vila da Luz, Belo Horizonte are -19.856027349999998, -43.89516383118321.  336
The geograpical coordinate of Jardim Atlântico, Belo Horizonte are -19.840769, -43.9922151.  337
The geograpical coordinate of Vila Formosa, Belo Horizonte are -19.998735349999997, -44.029693592543886.

The geograpical coordinate of Canadá, Belo Horizonte are -19.896521200000002, -43.94491367052375.  412
The geograpical coordinate of Antônio Ribeiro de Abreu, Belo Horizonte are -19.8314454, -43.8997695.  413
The geograpical coordinate of Satélite, Belo Horizonte are -19.8129804, -43.9481047.  414
The geograpical coordinate of Vila Antena Montanhês, Belo Horizonte are -19.8963971, -43.9859864.  415
The geograpical coordinate of Grotinha, Belo Horizonte are -19.85051765, -43.89231111805725.  416
The geograpical coordinate of Cônego Pinheiro, Belo Horizonte are -19.919032950000002, -43.90940352446526.  417
The geograpical coordinate of Chácara Leonina, Belo Horizonte are -19.954873749999997, -43.96042496060794.  418
The geograpical coordinate of Vila Madre Gertrudes III, Belo Horizonte are -19.9769783, -44.0185469.  419
The geograpical coordinate of Vila São Gabriel, Belo Horizonte are -19.846533700000002, -43.91231059704215.  420
The geograpical coordinate of Vila Paraíso, Belo Horizont

The geograpical coordinate of Guanabara, Belo Horizonte are -19.9701771, -44.1376229.  498
The geograpical coordinate of Lagoa da Pampulha, Belo Horizonte are -19.8533828, -43.97478514927906.  499
The geograpical coordinate of Sumaré, Belo Horizonte are -19.8875873, -43.9654843.  500
Vila Vera Cruz I wasn't found on Nominatim
Vila Vera Cruz II wasn't found on Nominatim


In [146]:
df

,#,Zone,Regional,Neighborhood,Population,Size(km²),latitude,longitude
0,1,ZONA LESTE,Leste,Sagrada Família,34 395,"2,291\n",-20.1964,-44.1078
1,2,ZONA SUL,Oeste,Buritis,29 374,"3,823\n",-19.9766,-43.9674
2,3,ZONA OESTE,Noroeste,Padre Eustáquio,28 773,"2,969\n",-19.9143,-43.9794
3,4,ZONA OESTE,Barreiro,Lindéia,25 231,"2,012\n",-19.9774,-44.0506
4,5,ZONA NORTE,Pamp./ Venda Nova,Santa Mônica,23 883,"2,178\n",-19.8246,-43.9775
...,...,...,...,...,...,...,...,...
498,483,ZONA LESTE,Nordeste,Guanabara,0,"0,002\n",-19.9702,-44.1376
499,484,ZONA NORTE,Pampulha,Lagoa da Pampulha,0,"3,138\n",-19.8534,-43.9748
500,485,ZONA NORTE,Noroeste,Sumaré,0,"0,149\n",-19.8876,-43.9655
501,486,ZONA LESTE,Leste,Vila Vera Cruz I,0,"0,008\n",,


In [147]:
df.drop(df[df['latitude']==''].index,inplace=True)
df.reset_index(drop=True,inplace=True)
df

,#,Zone,Regional,Neighborhood,Population,Size(km²),latitude,longitude
0,1,ZONA LESTE,Leste,Sagrada Família,34 395,"2,291\n",-20.1964,-44.1078
1,2,ZONA SUL,Oeste,Buritis,29 374,"3,823\n",-19.9766,-43.9674
2,3,ZONA OESTE,Noroeste,Padre Eustáquio,28 773,"2,969\n",-19.9143,-43.9794
3,4,ZONA OESTE,Barreiro,Lindéia,25 231,"2,012\n",-19.9774,-44.0506
4,5,ZONA NORTE,Pamp./ Venda Nova,Santa Mônica,23 883,"2,178\n",-19.8246,-43.9775
...,...,...,...,...,...,...,...,...
479,481,ZONA NORTE,Pampulha,Bispo de Maura,0,"0,017\n",-19.8281,-44.0164
480,482,ZONA NORTE,Pampulha,Campus UFMG,0,"3,85\n",-19.868,-43.9644
481,483,ZONA LESTE,Nordeste,Guanabara,0,"0,002\n",-19.9702,-44.1376
482,484,ZONA NORTE,Pampulha,Lagoa da Pampulha,0,"3,138\n",-19.8534,-43.9748


In [119]:
# create map of New York using latitude and longitude values
address = 'Belo Horizonte, MG'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_BH = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Zone'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_BH)  
    
map_BH

In [148]:
CLIENT_ID = 'UOSOUPHZSZ31RZYIACFNWILJ4U1E4HJPT51BGJEXQUKCUJSW' # your Foursquare ID
CLIENT_SECRET = 'ZRXAYW4AVLVJAOAOQP3QS2M1CLFUF5SGCLB5GUQYAM10R2MG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UOSOUPHZSZ31RZYIACFNWILJ4U1E4HJPT51BGJEXQUKCUJSW
CLIENT_SECRET:ZRXAYW4AVLVJAOAOQP3QS2M1CLFUF5SGCLB5GUQYAM10R2MG


In [149]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [154]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    return(nearby_venues)

In [159]:
BH_venues = getNearbyVenues(names=df['Neighborhood'],latitudes=df['latitude'],longitudes=df['longitude'])
BH_venues

Sagrada Família


KeyError: 'groups'

In [160]:
requests.get(url).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)